# Block 2: Lexical Level. SMS Spam Filtering.
Jordi Armengol - Joan Llop

## Data
We start by downloading the provided dataset.

Source: [UCI repository](https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection)
It consists of 5574 instances of SMS messages (aggregated from different sources) belonging to either the class 'ham' or the class 'spam'.

In [1]:
!wget -nc https://gebakx.github.io/ihlt/b2/resources/smsspamcollection.zip

--2019-11-06 23:50:20--  https://gebakx.github.io/ihlt/b2/resources/smsspamcollection.zip
Resolving gebakx.github.io (gebakx.github.io)... 185.199.111.153, 185.199.110.153, 185.199.109.153, ...
Connecting to gebakx.github.io (gebakx.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 203415 (199K) [application/zip]
Saving to: ‘smsspamcollection.zip’

smsspamcollection.z 100%[===================>] 198,65K  --.-KB/s    in 0,05s   

2019-11-06 23:50:21 (3,66 MB/s) - ‘smsspamcollection.zip’ saved [203415/203415]



In [2]:
!unzip smsspamcollection.zip

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [3]:
with open('SMSSpamCollection', 'r') as f:
    raw_data = f.readlines()

In [4]:
print('The dataset effectively has', len(raw_data), 'lines', 'with the following class distributions')
labeled_data = []
freqs = {}
for line in raw_data:
    label, text = line.split('\t')
    labeled_data.append((label, text))
    if label in freqs:
        freqs[label] += 1
    else:
        freqs[label] = 0
for key, value in freqs.items():
    percentage = 100*value/len(raw_data)
    print(key + ':', value, 'of', len(raw_data), '(' + "{0:.2f}".format(percentage) + '%)')

The dataset effectively has 5574 lines with the following class distributions
ham: 4826 of 5574 (86.58%)
spam: 746 of 5574 (13.38%)


The dataset is imbalanced, which poses a challenge when applying machine learning techniques. If we excesively optimize accuracy, perhaps we obtain low precision or recall values for the minority class. We will see this point with more detail later on.

In the particular case of spam, having a high precision for the spam class and low recall for the ham class should be the priority, because the cost of miss-classifying a legit message as spam is way higher than the opposite. However, we do not want to take this criterium to the extremes, because such classifier would be useless and actually behave like a constant predictor that always outputs 'ham'.

## Preprocessing

In order to ease the task to the machine learning algorithm and decrease noise introduced by some variance that might be irrelevant to this problem, we will, as suggested by the assignment:
    - Remove punctuation.
    - Convert characters to lowercase.

Ideally, we would like to take both punctuation and case into account, because perhaps texts with many capital letters and exclamation marks have higher probability of being spam. However, since we do not have a large database, in this case it would make the learning process more difficult.

Moreover, we will perform an additional pre-processing step: removing stop words, which are common words that most search engines are programmed to ignore. Examples of stop words are articles such as "the". Probably, these kind of words are introducing noise.

In addition, we will use lemmas instead of the original words. Lemmatization gives the canonical form of words, decreasing the variance introduced by morphology, and since we will be using representations that do not take into account the order of the words, like bag of words, decreasing the vocabulary and considering all the words with the same lemma as equal can ease the process of learning.

In [5]:
import nltk
import string
# nltk.download('punkt')
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

wnl = WordNetLemmatizer()
stopwords_set = set(stopwords.words('english'))
def remove_punctuation(token):
    res = ''
    for c in token:
        if c not in string.punctuation:
            res += c
    return res

def lemmatize(p):
    if p[1][0] in {'N','V'}:
        return wnl.lemmatize(p[0].lower(), pos=p[1][0].lower())
    return p[0]

preprocessed_data = []
for label, text in labeled_data:
    tokenized = nltk.word_tokenize(text.lower())
    tokenized = [remove_punctuation(tok) for tok in tokenized if tok not in stopwords_set]
    tokenized = list(filter(None, tokenized))
    pos_tags = pos_tag(tokenized)
    lemmas = [lemmatize(pair) for pair in pos_tags]
    preprocessed_data.append((label, tokenized, lemmas))
        

## Experiment design
The assignment suggests a "single validation (50% - 50%)" and a random shuffle. In this context, by validation we understand test, so we will have a train-test split of 50-50. We will provide a random seed in order to make the experiments reproducible. The test set will only be used for the final evaluation of the trained model.

In [6]:
import random

shuffled_preprocessed_data = preprocessed_data.copy()
random.seed(1)
random.shuffle(shuffled_preprocessed_data)
n = len(shuffled_preprocessed_data)
train, test = shuffled_preprocessed_data[:n//2], shuffled_preprocessed_data[n//2:]
len(train), len(test)

(2787, 2787)

## Text representation
The way of representing text is crucial for the application of machine learning techniques to natural language. Typically, each machine learning algorithm has an associated input format. In our case, we are keen on defining a custom kernel for sets in SVMs for the reasons we will see later on. Therefore, we will be working with sets, and the required text representation for our algorithm will be a bag of words, that is to say, a boolean vector codifying the presence of the indexed words. Bags of words encode the set of present words as a vector such that the present words will have its corresponding element set to true.

This representation has some caveats, since it does not account for the frequency of the words, for instance, unlike other representations. However, it is the required one for the algorithm of our choice, and perhaps in short texts frequency is not that important. Among the proposed representations, term-frequency times inverse document-frequency seems to be the most robust one, since it avoids the effect of common words.

In order to obtain a honest evaluation of the method, only words in the train set will be indexed. 

In [7]:
from functools import reduce
train_tokens = [tokens for label, tokens, lemmas in train]
flat_train_tokens = list(reduce(lambda x, y: x + y, train_tokens))
fdist_tokens = nltk.FreqDist(flat_train_tokens)
train_lemmas = [lemmas for label, tokens, lemmas in train]
flat_train_lemmas = list(reduce(lambda x, y: x + y, train_lemmas))
fdist_lemmas = nltk.FreqDist(flat_train_lemmas)
fdist_tokens, fdist_lemmas

(FreqDist({'u': 559,
           'need': 71,
           'presnts': 1,
           'always': 36,
           'bcz': 1,
           'cant': 35,
           'mis': 2,
           'love': 89,
           'jeevithathile': 1,
           'irulinae': 1,
           'neekunna': 1,
           'prakasamanu': 1,
           'sneham': 1,
           'prakasam': 1,
           'ennal': 1,
           'prabha': 2,
           'that': 2,
           'mns': 1,
           'islove': 1,
           'got': 120,
           'dont': 75,
           'wana': 6,
           'plan': 18,
           'trip': 13,
           'sometme': 1,
           'home': 89,
           'also': 35,
           'cos': 33,
           'lar': 21,
           'm': 190,
           'ba': 1,
           'dao': 1,
           'ok': 149,
           '1': 45,
           'pm': 5,
           'lor': 90,
           'never': 24,
           'ask': 52,
           'go': 151,
           'ah': 19,
           'said': 56,
           'would': 38,
           'fri': 11,
         

In [8]:
import numpy as np

def get_bow_from_freq(fdist, tokens):
    bow = []
    for key in fdist:
        if key in tokens:
            bow.append(True)
        else:
            bow.append(False)
    return bow

def build_mat(fdist, texts):
    mat = []
    for text in texts:
        bow = get_bow_from_freq(fdist, text)
        mat.append(bow)
    return np.array(mat)

X_train_lemmas = build_mat(fdist_lemmas, train_lemmas)
test_lemmas = [lemmas for label, tokens, lemmas in test]
X_test_lemmas = build_mat(fdist_lemmas, test_lemmas)
y_train = np.array([label for label, tokens, lemmas in train])
y_test = np.array([label for label, tokens, lemmas in test])

In [13]:
def print_results(y_test, y_pred):
    # TN: nº of mails classified as ham, FP: nº ham classified as spam, TP: nº spam classified as spam, FN: nº spam classified as ham
    ((TN, FP), (FN, TP)) = metrics.confusion_matrix(y_test, y_pred) 
    # print(metrics.classification_report(y_test, y_pred))
    presicion = TP/(TP + FP)
    recall = TP/(TP+FN)
    print('*'*35)
    print('* ', 'Confusion matrix'.center(29), ' *')
    print('*    \\ Predicted |', 'ham'.center(5), '|',  'spam'.center(5), '|*')
    print('* True\\', ' '*9, '_'*15, '*')
    print('* ', 'ham'.center(13), '|', str(TN).rjust(5), '|', str(FP).rjust(5), '|*')
    print('* ', 'spam'.center(13), '|', str(FN).rjust(5), '|', str(TP).rjust(5), '|*')
    print('*'*35)
    print()

## Machine learning: SVM with set kernel
As we have anticipated, from the suggested methods in the assignment, our machine learning algorithm of choice will be SVMs. In particular, we will define a custom kernel for determining set similarity between two given sets of tokens (ie. the respective bags of words of two given texts).

The reasons we have for using this method are the following:
- For the sake of learning and personal interests: We have used SVMs before in other subjects, but we had never used a custom kernel. In particular, we defined the set kernel as a theoretical exercise found in Bishop's Pattern Recognition and Machine Learning in the Machine Learning subject from the Computer Science degree at UPC, and demonstrated that it fulfills the kernel properties, but we had never actually coded it.
- Because SVMs are a powerful algorithm and we believe that can lead to good results in this case.

In a nutshell, an SVM (Support Vector Machines) is a supervised machine learning algorithm, tyically used for classification, that learns to define a decision boundary (specifically, a hyperplane) for separating the different classes. The algorithm itself is linear, but if the kernel, which is the function used for defining the similarity between two given instances, is not, then the model is non-linear. In our case, the kernel will be linear, but it is a custom one, because we want to define similarities between sets, not vectors.

If we encode a (finite) set as a boolean vector such that if the i-th element is defined as 'true' then the corresponding element of the set is present, we can compute the dot product between the boolean vectors of two instances. It will be equal to the number of elements that they have in common (so, the cardinality of the intersection). We can take 2 to the power of this cardinality instead, and the function will still be a valid kernel (see the properties of kernels in Bishop's Pattern Recognition and Machine Learning). This exponentiation can be useful for giving much more importance to the elements in common the more elements in common there are, instead of a linear growth.

Specifically, we define and implement the kernel set as:

$\kappa(S_1, S_2) = 2^{\vert S1 \cap S2\vert} = \sum_{j = 0}^{|U|-1} \mathcal{I}_x(u_j) \mathcal{I}_{x'}(u_j)$

with

$\mathcal{I}_x(u) :=
    \begin{cases}
      0, & \text{if}\ u \notin x\\
      1, & \text{otherwise}
    \end{cases}$

In [17]:
from sklearn import svm, metrics

def set_kernel(A, B):
    """During training, A and B are equal. In particular, they are the
    training data matrix, X (design matrix). In inference, A is the
    training X and B is the test X. We return a custom Gram matrix,
    in the sense that we define our own 'inner product' for boolean
    vectors representing sets. It can be interpreted as set similarity."""
    return np.exp2(np.dot(A, B.T))

classifier = svm.SVC(kernel=set_kernel, C=1.0)
classifier.fit(X_train_lemmas, y_train)
y_pred = classifier.predict(X_test_lemmas)
print('accuracy =', metrics.accuracy_score(y_true=y_test, y_pred=y_pred))
print()
print_results(y_test, y_pred)
print(metrics.classification_report(y_true=y_test, y_pred=y_pred))

accuracy = 0.902404018658

***********************************
*         Confusion matrix        *
*    \ Predicted |  ham  |  spam |*
* True\           _______________ *
*       ham      |  2411 |     0 |*
*       spam     |   272 |   104 |*
***********************************

              precision    recall  f1-score   support

         ham       0.90      1.00      0.95      2411
        spam       1.00      0.28      0.43       376

    accuracy                           0.90      2787
   macro avg       0.95      0.64      0.69      2787
weighted avg       0.91      0.90      0.88      2787



## Conclusions

The obtained accuracy score *seems* high, but it is **not**. Recall that the dataset was imbalanced and it is a binary classification problem, so a constant classifier that always returned 'ham' would have an accuracy of 0.8658%, which is the percentage of the majority class. In this sense, our results are not impressive, just slightly better than a constant classifier.

However, the precision and recall are way better than the ones we could expect from a constant classifier. In particular, they fulfill the requirements we said in the beginning. The precision of the spam class (and the recall of the ham class) is maximal, which means that no legit SMS is miss-classified as spam, which would have a higher cost than the opposite error. However, even with this condition, we still detect almost 30% of the SMSs messages, so our classifier is useful for filtering almost one third of the spam without removing by mistake any legit messages.

Note that our kernel implementation is very efficienty, because we have written pure Numpy vectorized code, without loops. It operates over the whole data matrix instead of going instance by instance. 

Notice that we have tried other approaches, like not using lemas and not discarding stop words, ans more simple machine learning algorithms, like K-NN or Naïve Bayes, but the results with these other approaches were not promising in some early experiments we performed in a subset of our training set. Optimizing the 'C' hyperparameter of the SVM (for penalizing the points outside the error) did not improve our results either.